# Langchain과 FAISS를 이용한 실습

### 설치 및 기본 설정(터미널에서 진행)

In [1]:
#!pip install langchain langchain-openai faiss-cpu

### OpenAI API 키 설정_환경변수엥서 API 키 가져오기

In [2]:
import os

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

## LangChain 기본 개념

### 언어모델 초기화

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

#모델 초기화
model = ChatOpenAI(model='gpt-4')

#모델에 메시지 전달
response = model.invoke([HumanMessage(content='안녕하세요, 무엇을 도와드릴까요?')])
print(response.content)

저는 AI입니다. 문장을 입력해주세요.


### 프롬프트 템플릿 사용하기

In [4]:
from langchain_core.prompts import ChatPromptTemplate

#시스템 메시지 설정
system_template = "Translate the following sentence from English to {language}:"

#사용자 텍스트 입력
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("user", "{text}")
])

#프롬프트 생성
result = prompt_template.invoke({"language": "korean", "text": "How are you?"})
print(result.to_messages())

[SystemMessage(content='Translate the following sentence from English to korean:', additional_kwargs={}, response_metadata={}), HumanMessage(content='How are you?', additional_kwargs={}, response_metadata={})]


## LangChain Expression Language (LCEL)로 체인 연결

In [7]:
from langchain_core.output_parsers import StrOutputParser

#응답을 파싱하는 파서 초기화
parser = StrOutputParser()

#템플릿, 모델, 파서를 체인으로 연결
chain = prompt_template | model | parser

#체인 실행
response = chain.invoke({"language": "korean", "text": "where is the library?"})
print(response)

도서관은 어디에 있나요?


## FAISS를 활용한 벡터 데이터베이스 구성 및 쿼리
FAISS는 벡터 유사성 검색을 위한 라이브러리.

***OpenAIEmbediings***로 텍스트를 벡터로 변환해 FAISS 인덱스에 저장.

### Step 1: OpenAI 임베딩 모델로 벡터 임베딩 생성

In [9]:
from langchain_openai import OpenAIEmbeddings

#OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

### Step 2: FAISS 인덱스 초기화

In [11]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

#Faiss 인덱스 생성
index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

### Step 3: 벡터 데이터베이스에 문서 추가

In [13]:
from langchain_core.documents import Document
from uuid import uuid4

#문서 생성
documents = [
    Document(page_content="LangChain을 사용해 프로젝트를 구축하고 있습니다!", metadata={"source":"tweet"}),
    Document(page_content="내일 날씨는 맑고 따뜻할 예정입니다.", metadata={"source":"news"}),
    Document(page_content="오늘 아침에는 팬케이크와 계란을 먹었어요.", metadata={"source":"personal"}),
    Document(page_content="주식 시장이 경기 침체 우려로 하락 중입니다.", metadata={"source":"news"}),
]

#고유 ID 셍성 및 문서 추가
uuids = [str(uuid4()) for _ in range(len(documents))]
vector_store.add_documents(documents=documents, ids=uuids)

['5bc9c16a-b0ed-4a75-ac9a-8ebf8b88bd2a',
 '25e33f71-4529-4a51-ac86-460bc5a46be9',
 'cc2dcde7-bdd8-4a46-83f4-b146205cb0c6',
 'f1f5cab4-c9ab-4460-aecf-6047299a5010']

### Step4: 벡터 데이터베이스 쿼리

In [16]:
#기본 유사성 검색
results = vector_store.similarity_search("내일 날씨는 어떨까요?", k=2, filter={"source":"news"})
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

#정수와 함꼐 유사성 검색
results_with_scores = vector_store.similarity_search_with_score("LangChain에 대해 이야기해주세요.", k=2, filter={"source":"tweet"})
for res, score in results_with_scores:
    print(f"* [SIM={score:.3f}] {res.page_content} [{res.metadata}]")

* 내일 날씨는 맑고 따뜻할 예정입니다. [{'source': 'news'}]
* 주식 시장이 경기 침체 우려로 하락 중입니다. [{'source': 'news'}]
* [SIM=0.159] LangChain을 사용해 프로젝트를 구축하고 있습니다! [{'source': 'tweet'}]


## RAG 체인에 FAISS 통합
RAG 체인을 구성하여 검색된 문서를 바탕으로 질문에 응답할 수 있도록 구성

### Step 1:  Retriever로 변환
FAISS를 retriever로 변환해 RAG 체인에서 사용

In [19]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k":1})

### Step 3: RAG 체인 생성
LangChain의 모델과 프롬프트를 연결하여 RAG 체인을 구성

In [24]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# 프롬프트 템플릿 정의
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question using only the following context."),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])


class DebugPassThrough(RunnablePassthrough):
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args, **kwargs)
        print("Debug Output:", output)
        return output
# 문서 리스트를 텍스트로 변환하는 단계 추가
class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config=None, **kwargs):  # config 인수 추가
        # context의 각 문서를 문자열로 결합
        context_text = "\n".join([doc.page_content for doc in inputs["context"]])
        return {"context": context_text, "question": inputs["question"]}

# RAG 체인에서 각 단계마다 DebugPassThrough 추가
rag_chain_debug = {
    "context": retriever,                    # 컨텍스트를 가져오는 retriever
    "question": DebugPassThrough()        # 사용자 질문이 그대로 전달되는지 확인하는 passthrough
}  | DebugPassThrough() | ContextToText()|   contextual_prompt | model

# 질문 실행 및 각 단계 출력 확인
response = rag_chain_debug.invoke("강사이름은?")
print("Final Response:")
print(response.content)

Debug Output: 강사이름은?
Debug Output: {'context': [Document(metadata={'source': 'news'}, page_content='내일 날씨는 맑고 따뜻할 예정입니다.')], 'question': '강사이름은?'}
Final Response:
The context does not provide information on the instructor's name.
